In [10]:
import openai
##put your key here
OPENROUTER_API_KEY = 'replace with your key'
YOUR_SITE_URL = 'www'
YOUR_APP_NAME = 'demo'
MODEL = 'openai/gpt-3.5-turbo-instruct'

# google/gemma-7b-it
# openai/gpt-3.5-turbo-instruct
# meta-llama/llama-2-70b-chat
# mistralai/mixtral-8x7b-instruct

# Define your constraints
max_tokens = 500  # Set your desired maximum word count here

# OpenRouter API base URL
openai.api_base = "https://openrouter.ai/api/v1"
openai.api_key = OPENROUTER_API_KEY


In [11]:
from os import getenv
from openai.error import APIError
import time

def use_model(model_name, content, max_retries=5):
    for i in range(max_retries):
        try:
            completion = openai.ChatCompletion.create(
            extra_headers={
                "HTTP-Referer": YOUR_SITE_URL, # Optional, for including your app on openrouter.ai rankings.
                "X-Title": YOUR_APP_NAME, # Optional. Shows in rankings on openrouter.ai.
            },
            model= model_name,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": content}
            ],
            max_tokens=max_tokens

            )
            return completion.choices[0].message.content
        except APIError as e:
            print(f"An error occurred: {e}. Retrying... (Attempt {i+1}/{max_retries})")
            time.sleep(2)  # Wait for 2 seconds before retrying
    print(f"Failed after {max_retries} attempts.")
    return None

In [12]:
import pandas as pd
import time
# Load model directly

# Replace 'your_file.csv' with the actual file path
file_path_image_description = 'dataForGeneration\BLIP2_Generate a thorough product description.csv'
file_path_text_description = 'dataForGeneration\sampled_products_10_each_img_front 1.csv'

# Read the CSV file into a DataFrame
df_image = pd.read_csv(file_path_image_description)
df_text = pd.read_csv(file_path_text_description)
merged_df = pd.merge(df_text, df_image, on='ItemSku', how='left')
merged_df['Generated Product Description'] = ''
merged_df['Generated Product Features'] = ''
model_name_for_saving = MODEL.replace("/","")
model_name_for_saving = model_name_for_saving.replace(":","")

#for products with image
for index, row in merged_df.iterrows():
    if pd.isna(row["MarketingName"]):
        name = row['ProductName']
    else:
        name = row['MarketingName']
    for index, col in enumerate(["create one-paragraph attractive product description", "create bullet-point product features"]):
        if (not pd.isna(row["MarketingDetails"])) and (not pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product with less than 500 words. Please be concise and only give me the content. Don\' mention things about image.: 1. Product Name: {name} 2. Product Details: {row["MarketingDetails"]} 3. Product Features: {row["MarketingFeatures"]} 4. Product Image Description: {row["UnconditionalCaption"]}'
        elif (pd.isna(row["MarketingDetails"])) and (not pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product with less than 500 words. Please be concise and only give me the content. Don\' mention things about image.: 1. Product Name: {name} 2. Product Features: {row["MarketingFeatures"]} 3. Product Image Description: {row["UnconditionalCaption"]}'
        elif (not pd.isna(row["MarketingDetails"])) and (pd.isna(row["MarketingFeatures"])):
            input_text = f'Please help me {col} for this product with less than 500 words. Please be concise and only give me the content. Don\' mention things about image.: 1. Product Name: {name} 2. Product Details: {row["MarketingDetails"]} 3. Product Image Description: {row["UnconditionalCaption"]}'
        else:
            input_text = f'Please help me {col} for this product with less than 500 words. Please be concise and only give me the content. Don\' mention things about image.: 1. Product Name: {name} 2. Product Image Description: {row["UnconditionalCaption"]}'

        output = use_model(MODEL, input_text)
        print(f"ItemSku: {row['ItemSku']}\nOutput {index}:\n{output}")

        if index == 0:
            merged_df.loc[merged_df['ItemSku'] == row['ItemSku'],['Generated Product Description']] = output
        else:
            merged_df.loc[merged_df['ItemSku'] == row['ItemSku'],['Generated Product Features']] = output

    
    merged_df.to_csv(f"Generated_text_{model_name_for_saving}_0305.csv", index= False)
        

    



ItemSku: 19283315462
Output 0:

The Yankee Candle Signature Collection Small Tumbler Vanilla Cupcake is a decadent combination of rich vanilla, hints of zesty lemon, and buttery icing. This fragrance is perfect for those who love the sweet and comforting aroma of vanilla cupcakes. The top notes of vanilla icing and malted sugar create a delectable initial impression, while the middle notes of cake batter and chocolate add depth to the scent. The base notes of cocoa and vanilla bean leave a lasting impression that will keep you craving more. The unique brushed tin lid of this candle can also double as a coaster, making it a convenient addition to any room. With a clean-burning cotton wick and a premium soy-wax blend, you can enjoy the sweet scent of vanilla without any worries. The hand-illustrated label and vibrant wax colors make this candle a beautiful addition to any home décor. Get ready to indulge your senses with the Yankee Candle Signature Collection Small Tumbler Vanilla Cupcak